In [1]:
import pandas as pd
import numpy as np
import branca
import folium
import csv , json, os
import geocoder
import ipywidgets
import branca.colormap as cm

from folium.features import CustomIcon
from PIL import Image
from folium.plugins import MiniMap
from folium import plugins
from folium.plugins import Search
from folium.plugins import MarkerCluster
from branca.colormap import linear


with open('NCR_barangay.geojson', 'r') as json_file:
    ncr_bar = json.load(json_file)

with open('ncr_city.geojson', 'r') as json_file:
    ncr_city_geojson = json.load(json_file)
ncr_city = pd.read_csv('ncr_city.csv')
lst = ['latitude','longitude','altitude']
ncr_city.drop(lst, axis=1, inplace=True)


cols = cm.LinearColormap(
    ['yellow','orange','red'],
    vmin=0, vmax=350
)
cols1 = cm.LinearColormap(
    ['orange','red'],
    vmin=0, vmax=28
).to_step(26)

ncr_dict = ncr_city.set_index('City')['ConfirmedCases']

    
    

df = pd.read_csv('NCR_BARANGAY.csv')
df.drop('Unnamed: 0', axis=1, inplace=True)
df.Confirmed_Cases = df.Confirmed_Cases.astype(int)



#colormap = linear.Reds_09.scale(
#    df.Confirmed_Cases.min(),
#    df.Confirmed_Cases.max())
#df_dict = df.set_index('Barangay')['Confirmed_Cases']


colormap = cm.LinearColormap(
    ['green','yellow','orange','red'],
    vmin=0, vmax=28
)



m = folium.Map(location=(14.6091, 121.0223), zoom_start=12)
barangay_geo = folium.GeoJson(
    ncr_bar,
    name='Barangay View',
   # show=False,
    #0.7
    tooltip=folium.GeoJsonTooltip(fields=['Barangay','Confirmed_Cases'],
                                  aliases=['Barangay: ','Confirmed Cases:'],
                                    sticky=False,
                                 ),
    highlight_function=lambda x: {'fillOpacity':0.1},
    style_function=lambda feature: {
        'fillColor': 'green' if feature['properties']['Confirmed_Cases'] == 0 else cols1(feature['properties']['Confirmed_Cases']),
        'fillOpacity': 0.7,
        'color': 'black',
        'weight': 1.2}).add_to(m)  




city_geo = folium.GeoJson(
    ncr_city_geojson,
    name='City View',
    show=False,
    #0.7
    tooltip=folium.GeoJsonTooltip(fields=['City','ConfirmedCases'],
                                  aliases=['City: ','Confirmed Cases:'],
                                    sticky=False,),
    highlight_function=lambda x: {'fillOpacity':0.1},
    style_function=lambda feature: {
        'fillColor':cols(ncr_dict[feature['properties']['City']]),
        'fillOpacity': 0.7,
        'color': 'black',
        'weight': 1.2}).add_to(m)  

Confirmed_geo = folium.GeoJson(
    ncr_bar,
    name='Barangay with confirmed and no case/s',
    show=False,
 
    #0.7
    tooltip=folium.GeoJsonTooltip(fields=['Barangay','Confirmed_Cases'],
                                  aliases=['Barangay: ','Confirmed Cases:'],
                                    sticky=False,),
    
    highlight_function=lambda x: {'fillOpacity':0.1},
    style_function=lambda feature: {
        'fillColor': 'red' if feature['properties']['Confirmed_Cases'] > 0 else 'green',
        'fillOpacity': 0.7,
        'color': 'black',
        'weight': 1.2}).add_to(m) 
    
citysearch = Search(
    layer=barangay_geo,
    geom_type='Polygon',
    placeholder='Search for Barangay',
    collapsed=True,
    search_label='Barangay',
    weight=3
).add_to(m)

colormap.caption = 'Confirmed Cases of Covid-19'
colormap.add_to(m)

plugins.Fullscreen(
position='topright',
title='Expand me',
title_cancel='Exit me',
force_separate_button=True).add_to(m)

folium.LayerControl(collapsed=True, autoZIndex=False).add_to(m)
plugins.LocateControl(auto_start=True).add_to(m)
m


In [9]:
m.save('index.html')